# Data Drift Workflow



In [ ]:
import nannyml as nml
import pandas as pd

In [ ]:
reference, analysis, analysis_gt = nml.load_synthetic_sample()

In [ ]:
md = nml.extract_metadata(data = reference, model_name='wfh_predictor')

In [ ]:
print(md.print())
# md.print()

In [ ]:
md.timestamp_column_name = 'timestamp'
# md.prediction_column_name = 'y_pred_proba'
md.ground_truth_column_name = 'work_home_actual'

In [ ]:
print(md.print())

In [ ]:
univariate_calculator = nml.UnivariateStatisticalDriftCalculator(model_metadata=md, chunk_size=5000)

In [ ]:
univariate_calculator.fit(reference_data=reference)

In [ ]:
fdata = pd.concat([reference, analysis], ignore_index=True)
fdata

In [ ]:
univariate_results = univariate_calculator.calculate(data=fdata)

In [ ]:
univariate_results_columns = list(univariate_results.columns)
univariate_results_columns

In [ ]:
univariate_results#.iloc[-5:, :9]

In [ ]:
ranking = nml.AlertCountRanking()

In [ ]:
ranked_features_drifted = ranking.rank(univariate_results)

In [ ]:
ranked_features_drifted

In [ ]:
rcerror_calculator = nml.DataReconstructionDriftCalculator(model_metadata=md, chunk_size=5000)

In [ ]:
rcerror_calculator.fit(reference_data=reference)

In [ ]:
rcerror_results = rcerror_calculator.calculate(data=fdata)
rcerror_results

In [ ]:
plots = nml.DriftPlots(univariate_calculator)

In [ ]:
for itm in md.features:

    fig = plots.plot_univariate_statistical_drift(univariate_results, metric='statistic', feature_label=itm.label)
    fig.show()
    fig.write_image(file=f"drift-guide-{itm.label}.svg")

In [ ]:
for itm in md.continuous_features:

    fig = plots.plot_continuous_feature_distribution_over_time(
        data=pd.concat([reference, analysis], ignore_index=True),
        drift_results=univariate_results,
        feature_label=itm.label
    )
    fig.show()
    fig.write_image(file=f"drift-guide-joyplot-{itm.label}.svg")

In [ ]:
for itm in md.categorical_features:

    fig = plots.plot_categorical_feature_distribution_over_time(
        data=pd.concat([reference, analysis], ignore_index=True),
        drift_results=univariate_results,
        feature_label=itm.label
    )
    fig.show()
    fig.write_image(file=f"drift-guide-stacked-{itm.label}.svg")

In [ ]:
fig = plots.plot_data_reconstruction_drift(rcerror_results)
fig.show()
fig.write_image(file=f"drift-guide-multivariate.svg")